In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
import json
from dotenv import load_dotenv
from loguru import logger

print(load_dotenv())

sys.path.insert(1, os.path.join( os.getcwd(), ".." ) )
from src.llm import multi_llm
from src.utils.notebook_vs_util.vs_path import vs_notebook_name, number_log

from src.utils.ea.exp_funcs import pricing

notebook_name = vs_notebook_name()
n = number_log(".\logs")
logger.add(f".\logs\{notebook_name}_{n}.log")

n


True


2025-03-30 22:13:26.027 | INFO     | src.utils.openaiwrapper.openai_wrapper:<module>:17 - Update cost from langchain
2025-03-30 22:13:27.146 | INFO     | src.utils.openaiwrapper.get_openai_cost:extract_and_save_model_cost_data:106 - Model cost data saved to c:\Users\caanp\OneDrive\MBA\Repositorio\GenAI-MBA\src\utils\openaiwrapper\model_costs.json
2025-03-30 22:13:29.205 | DEBUG    | src.utils.notebook_vs_util.vs_path:vs_notebook_name:19 - Extract the notebook name
2025-03-30 22:13:29.209 | DEBUG    | src.utils.notebook_vs_util.vs_path:vs_notebook_name:22 - Extract the notebook name: 02.llm_openai_gemini
2025-03-30 22:13:29.210 | DEBUG    | src.utils.notebook_vs_util.vs_path:vs_notebook_name:19 - Extract the notebook name
2025-03-30 22:13:29.211 | DEBUG    | src.utils.notebook_vs_util.vs_path:vs_notebook_name:22 - Extract the notebook name: 02.llm_openai_gemini
2025-03-30 22:13:29.214 | DEBUG    | src.utils.notebook_vs_util.vs_path:number_log:65 - The next log number determined is: 5


5

## Promp enginering

Prompt Techniques: 
- Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models: https://arxiv.org/pdf/2305.04091.pdf
- Prompt page: https://www.promptingguide.ai/techniques
- ToT: https://github.com/dave1010/tree-of-thought-prompting
- PAL: https://www.promptingguide.ai/techniques/pal
- CoT: https://www.promptingguide.ai/techniques/cot

## Discussion
- https://github.com/holarissun/PanelGPT

# 01. Data

Carregando os dados de amostra com 1%

In [2]:
sample_1 = pd.read_parquet("./../Dados/sample_1_Meli.parquet")
sample_1.shape

(978, 21)

In [3]:
only_emoji_reviews = sample_1.query("with_emojis==True and qtd_words==1").reset_index(drop=True)
only_emoji_reviews.shape

(5, 21)

In [4]:
only_emoji_reviews.head(5)

,reviewId,at,content,score,reviewCreatedVersion,replyContent,repliedAt,appVersion,appId,sentiment,...,qtd_characters,qtd_words,list_emojis,qtd_emojis,with_emojis,input_$_gpt_35,input_$_gpt_4o,input_$_gpt_4o-mini,input_$_gpt_4_turbo,input_$_gpt_4
0,91f1a738-05f7-4ca8-9c7a-87e5bccf2a96,2021-05-02 20:48:34,👍,3,10.154.2,None,None,10.154.2,com.mercadolibre,1,...,1,1,[👍],1,True,0.000002,0.000015,4.500000e-07,0.00003,0.00009
1,cc1b92e8-583f-4292-a1fd-7c950f66469c,2021-06-03 19:04:21,Bom😇,3,10.159.4,None,None,10.159.4,com.mercadolibre,1,...,4,1,[😇],1,True,0.000002,0.000020,6.000000e-07,0.00004,0.00012
2,710622ac-a057-4058-93f2-a1bd586ab23b,2023-09-10 14:59:14,❣️,5,10.277.4,None,None,10.277.4,com.mercadolibre,2,...,2,1,[❣],1,True,0.000002,0.000015,4.500000e-07,0.00003,0.00009
3,d468e35e-3ba2-4c9f-8552-a216293da1a9,2023-09-24 09:47:48,👍👍👍👍,5,10.279.2,None,None,10.279.2,com.mercadolibre,2,...,4,1,"[👍, 👍, 👍, 👍]",4,True,0.000006,0.000060,1.800000e-06,0.00012,0.00036
4,a0f9c333-adf5-4264-8d82-25fcebb6ee23,2023-09-16 21:15:00,😍😍,5,10.278.2,None,None,10.278.2,com.mercadolibre,2,...,2,1,"[😍, 😍]",2,True,0.000002,0.000020,6.000000e-07,0.00004,0.00012


# 0.2 Prompts and Models

In [5]:
app = "mercado livre"

prompt_tot = """3 experts are discussing the question with a panel discussion, trying to solve it step by step, and make sure the result is correct and avoid penalty.

The question is...                        
            
```{review}```

What is the sentiment of the {app} app review? Positive, negative, or neutral?

Output the result as a JSON object, including the `Sentimento` and `Justificativa`. Answer in portuguese."""

zero_shot_prompt =  """
"Analyze the sentiment of the following {app} app review text, classifying it as positive, negative, or neutral.

```{review}```

Output the result as a JSON object, including the `Sentimento` and `Justificativa`. Answer in portuguese."
"""

In [6]:
prompts = {
            "prompt_tot":prompt_tot,
            "zero_shot_prompt":zero_shot_prompt
           }

models = ["gpt-3.5-turbo","gpt-4o", "gpt-4o-mini"]
# models = ["gpt-3.5-turbo","gpt-4o","gemini-1.5-pro"]

In [8]:
from openai import OpenAI

llm_op  = OpenAI()

for model in models:
        saida = llm_op.chat.completions.create(messages=[
                                        {"role": "system", "content": ""},
                                        {"role": "user", "content": "hola"},
                                        ],
                                model = model )
        
        logger.info(f"modelo: {model} - version: {saida.model}")

2025-03-30 22:14:53.866 | INFO     | __main__:<module>:12 - modelo: gpt-3.5-turbo - version: gpt-3.5-turbo-0125
2025-03-30 22:14:54.325 | INFO     | __main__:<module>:12 - modelo: gpt-4o - version: gpt-4o-2024-08-06
2025-03-30 22:14:54.808 | INFO     | __main__:<module>:12 - modelo: gpt-4o-mini - version: gpt-4o-mini-2024-07-18


# 0.3 LLM and Sentiment Analysis

## 03.1 LLM

Class to use Gemini and OpenAI.

- OpenAI is paid. The cost depends on the model.
- Gemini is free, but it has quota limit per model.

In [14]:
llm  = multi_llm()

## 03.2 Sentiment Analysis

- The execution 2 spends more time than execution 1


In [139]:
df = only_emoji_reviews[['reviewId', 'content', 'score']]

sample_1.shape

(978, 20)

In [17]:
from pandas import DataFrame

def execute_models_prompts(df:DataFrame , models:list, prompts: dict, app:str = "Mercado Livre"):
    
    df_ = df.copy()

    for n, model in enumerate(models, start=1):
        
        logger.debug(f"{n}/{len(models)}. {model}")
        
        for k, prompt_name__prompt in enumerate(prompts.items(), start=1):
            
            prompt_name, prompt  = prompt_name__prompt
            
            logger.debug(f"\t{k}/{len(prompts)}. {prompt_name}")

        
            for m, idx__row in enumerate(df_.iterrows(), start=1):
                
                idx, row = idx__row
                
                # print(f"\t\t{m}/{len(df_)}.")
                
                
                review = row["content"]
                prompt_anl = prompt.format(app = app, review = review )
                
                try:
                
                    result, usage = llm.response(model = model , user_message = prompt_anl, json_output=True)
                    
                    j_result = json.loads(result)
                    
                    ## Result
                    df_.loc[idx,f"Sentiment__{prompt_name}__{model}"] = j_result["Sentimento"]
                    df_.loc[idx,f"Justificativa__{prompt_name}__{model}"] = j_result["Justificativa"]
                    ## Cost
                    df_.loc[idx,f"usage_completion_tokens__{prompt_name}__{model}"] = usage["completion_tokens"]
                    df_.loc[idx,f"usage_prompt_tokens__{prompt_name}__{model}"] = usage["prompt_tokens"]
                    df_.loc[idx,f"usage_total_tokens__{prompt_name}__{model}"] = usage["total_tokens"]
                    df_.loc[idx,f"usage_total_cost__{prompt_name}__{model}"] = usage["total_cost"]
                
                except Exception as e:
                    logger.error(f"Error: {e}")
                
    return df_

In [18]:
df_r = execute_models_prompts( sample_1 , models[::-1] , prompts)

2025-01-27 22:09:19.390 | DEBUG    | __main__:execute_models_prompts:9 - 1/3. gpt-4o-mini
2025-01-27 22:09:19.391 | DEBUG    | __main__:execute_models_prompts:15 - 	1/2. prompt_tot
2025-01-27 22:35:23.167 | DEBUG    | __main__:execute_models_prompts:15 - 	2/2. zero_shot_prompt
2025-01-27 23:38:49.635 | DEBUG    | __main__:execute_models_prompts:9 - 2/3. gpt-4o
2025-01-27 23:38:49.635 | DEBUG    | __main__:execute_models_prompts:15 - 	1/2. prompt_tot
2025-01-28 09:36:31.582 | DEBUG    | __main__:execute_models_prompts:15 - 	2/2. zero_shot_prompt
2025-01-28 14:11:12.444 | DEBUG    | __main__:execute_models_prompts:9 - 3/3. gpt-3.5-turbo
2025-01-28 14:11:12.444 | DEBUG    | __main__:execute_models_prompts:15 - 	1/2. prompt_tot
2025-01-28 17:07:24.593 | ERROR    | __main__:execute_models_prompts:44 - Error: Unterminated string starting at: line 3 column 22 (char 53)
2025-01-28 17:16:46.702 | DEBUG    | __main__:execute_models_prompts:15 - 	2/2. zero_shot_prompt


In [134]:
df_r.to_parquet(f"./../Dados/result_analyzer_{n}.parquet", index=False)

# 00. Read

In [143]:
df_r = pd.read_parquet(f"./../Dados/result_analyzer_{n}.parquet")
n

2

# 04. Cost

In [144]:
by_model_1, total_1 = pricing(df_r, [x for x in df_r.columns if "total_cost" in x] )
by_model_1

,tokens ($USD),lines
modelo,,
usage_total_cost__prompt_tot__gpt-3.5-turbo,0.183303,978
usage_total_cost__prompt_tot__gpt-4o,1.005578,978
usage_total_cost__prompt_tot__gpt-4o-mini,0.052382,978
usage_total_cost__zero_shot_prompt__gpt-3.5-turbo,0.164818,978
usage_total_cost__zero_shot_prompt__gpt-4o,0.871123,978
usage_total_cost__zero_shot_prompt__gpt-4o-mini,0.047547,978


In [145]:
logger.info(f"\n\n{by_model_1}\n\n")

2025-01-28 21:16:09.511 | INFO     | __main__:<module>:1 - 

                                                   tokens ($USD)  lines
modelo                                                                 
usage_total_cost__prompt_tot__gpt-3.5-turbo             0.183303    978
usage_total_cost__prompt_tot__gpt-4o                    1.005578    978
usage_total_cost__prompt_tot__gpt-4o-mini               0.052382    978
usage_total_cost__zero_shot_prompt__gpt-3.5-turbo       0.164818    978
usage_total_cost__zero_shot_prompt__gpt-4o              0.871123    978
usage_total_cost__zero_shot_prompt__gpt-4o-mini         0.047547    978




In [146]:
logger.info(f"\n\n{total_1}\n\n")
# total_1

2025-01-28 21:16:09.756 | INFO     | __main__:<module>:1 - 

tokens ($USD)       2.324748
lines            5868.000000
dtype: float64




## Evaluation

In [147]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('max_colwidth',None)

In [148]:
df_r["Sentiment"] = df_r["sentiment"].apply(lambda x: "Negativo" if x==0 else "Neutro" if x==1 else "Positivo")
df_r["Sentiment"].value_counts()



Sentiment
Positivo    356
Negativo    320
Neutro      302
Name: count, dtype: int64

In [149]:
sentiment_cols = [x for x in df_r.columns if "Sentiment_" in x]

for sent_col in sentiment_cols:
    
    df_r[f"{sent_col}_pred"] = df_r[sent_col].apply(lambda x: 0 if x=="Negativo" else 1 if x=="Neutro" else 2)
    

In [150]:
results = [x for x in df_r.columns if "Sentiment__" and "pred" in x]
results

['Sentiment__prompt_tot__gpt-4o-mini_pred',
 'Sentiment__zero_shot_prompt__gpt-4o-mini_pred',
 'Sentiment__prompt_tot__gpt-4o_pred',
 'Sentiment__zero_shot_prompt__gpt-4o_pred',
 'Sentiment__prompt_tot__gpt-3.5-turbo_pred',
 'Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred']

In [151]:


def show_confusion_matrix(confusion_matrix, label):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('Sentimento Verdadeiro')
  plt.xlabel('Sentimento Predito')
  
  plt.tight_layout()
  
  plt.savefig(f"./../Results/{label}.png")
  
  plt.clf()
  plt.close()


In [152]:
['Sentiment__prompt_tot__gpt-3.5-turbo_pred',
 'Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred',
 'Sentiment__prompt_tot__gpt-4o_pred',
 'Sentiment__zero_shot_prompt__gpt-4o_pred']

['Sentiment__prompt_tot__gpt-3.5-turbo_pred',
 'Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred',
 'Sentiment__prompt_tot__gpt-4o_pred',
 'Sentiment__zero_shot_prompt__gpt-4o_pred']

In [153]:
class_names = ["Negativo","Neutro","Positivo"]

In [154]:
for resultado in results:

  cm = confusion_matrix(df_r["sentiment"], df_r[resultado])

  
  df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
    
  show_confusion_matrix(df_cm, label=resultado + f"__round_{n}")

In [155]:
for resultado in results:

    print(resultado)
    print("\n")
    print(classification_report(df_r["sentiment"],  df_r[resultado] , target_names = class_names ))
    print("===================================================")

Sentiment__prompt_tot__gpt-4o-mini_pred


              precision    recall  f1-score   support

    Negativo       0.50      0.95      0.65       320
      Neutro       0.50      0.03      0.06       302
    Positivo       0.74      0.72      0.73       356

    accuracy                           0.58       978
   macro avg       0.58      0.57      0.48       978
weighted avg       0.59      0.58      0.50       978

Sentiment__zero_shot_prompt__gpt-4o-mini_pred


              precision    recall  f1-score   support

    Negativo       0.34      0.26      0.29       320
      Neutro       0.41      0.03      0.06       302
    Positivo       0.42      0.84      0.56       356

    accuracy                           0.40       978
   macro avg       0.39      0.38      0.30       978
weighted avg       0.39      0.40      0.32       978

Sentiment__prompt_tot__gpt-4o_pred


              precision    recall  f1-score   support

    Negativo       0.55      0.94      0.69       320
  

In [156]:
for resultado in results:

    logger.info(f"{resultado}\n")
    logger.info(classification_report(df_r["sentiment"],  df_r[resultado] , target_names = class_names ))
    logger.info(f"=====================================")

2025-01-28 21:16:16.546 | INFO     | __main__:<module>:3 - Sentiment__prompt_tot__gpt-4o-mini_pred

2025-01-28 21:16:16.562 | INFO     | __main__:<module>:4 -               precision    recall  f1-score   support

    Negativo       0.50      0.95      0.65       320
      Neutro       0.50      0.03      0.06       302
    Positivo       0.74      0.72      0.73       356

    accuracy                           0.58       978
   macro avg       0.58      0.57      0.48       978
weighted avg       0.59      0.58      0.50       978

2025-01-28 21:16:16.562 | INFO     | __main__:<module>:5 - =====================================
2025-01-28 21:16:16.562 | INFO     | __main__:<module>:3 - Sentiment__zero_shot_prompt__gpt-4o-mini_pred

2025-01-28 21:16:16.579 | INFO     | __main__:<module>:4 -               precision    recall  f1-score   support

    Negativo       0.34      0.26      0.29       320
      Neutro       0.41      0.03      0.06       302
    Positivo       0.42      0.84  

In [157]:
# df_r.query(f"sentiment==1 and `Sentiment__zero_shot_prompt__gpt-3.5-turbo`=='Positivo'")[["content","sentiment","Sentiment__zero_shot_prompt__gpt-3.5-turbo","Justificativa__zero_shot_prompt__gpt-3.5-turbo"]]

In [158]:
# print(classification_report(df_r["sentiment"],  df_r["Sentiment__zero_shot_prompt__gpt-4o_pred"] , target_names = class_names ) )

In [52]:
# print(classification_report(df_r["sentiment"],  df_r["Sentiment__prompt_tot__gpt-4o_pred"] , target_names = class_names ))

In [53]:
# print(classification_report(df_r["sentiment"],  df_r["Sentiment__zero_shot_prompt__gpt-3.5-turbo_pred"] , target_names = class_names ))

In [54]:
# print(classification_report(df_r["sentiment"],  df_r["Sentiment__prompt_tot__gpt-3.5-turbo_pred"] , target_names = class_names ))

In [55]:
# [x for x in df_r.columns if "Sentiment_" in x]